In [0]:
!pip install -q tensorflow-gpu==2.0.0-alpha0

In [0]:
!pip install sacrebleu # https://github.com/mjpost/sacreBLEU

In [0]:
!unzip spa-eng

Archive:  spa-eng.zip
replace spa-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
!ls spa-eng

_about.txt  spa.txt


In [0]:
import numpy as np
import re
import sacrebleu
import tensorflow as tf
import time
import unicodedata

In [5]:
tf.__version__

'2.0.0-alpha0'

In [6]:
with open('spa-eng/spa.txt') as f:
  sentences=[]
  for _ in range(5000):
    tmp=f.readline()
    sentences.append(list(reversed(tmp.split('\t'))))
sentences[:10]

[['Ve.\n', 'Go.'],
 ['Vete.\n', 'Go.'],
 ['Vaya.\n', 'Go.'],
 ['Váyase.\n', 'Go.'],
 ['Hola.\n', 'Hi.'],
 ['¡Corre!\n', 'Run!'],
 ['Corred.\n', 'Run.'],
 ['¿Quién?\n', 'Who?'],
 ['¡Fuego!\n', 'Fire!'],
 ['¡Incendio!\n', 'Fire!']]

In [0]:
def preprocess(s):
  # for details, see https://www.tensorflow.org/alpha/tutorials/sequences/nmt_with_attention
  s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)
  s = s.strip()
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  s = '<start> ' + s + ' <end>'
  return s

In [8]:
print("Original:", sentences[0])
sentences = [(preprocess(source), preprocess(target)) for (source, target) in sentences]
print("Preprocessed:", sentences[0])

Original: ['Ve.\n', 'Go.']
Preprocessed: ('<start> Ve . <end>', '<start> Go . <end>')


In [0]:
#pair english & spanish
source_sentences, target_sentences = list(zip(*sentences))

In [10]:
source_sentences[0],target_sentences[0]

('<start> Ve . <end>', '<start> Go . <end>')

In [11]:
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
source_tokenizer.fit_on_texts(source_sentences)
source_data = source_tokenizer.texts_to_sequences(source_sentences)
print("Sequence:", source_data[0])
source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='post')
print("Padded:", source_data[0])

Sequence: [1, 71, 3, 2]
Padded: [ 1 71  3  2  0  0  0  0  0  0  0  0]


In [0]:
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
target_tokenizer.fit_on_texts(target_sentences)
target_data = target_tokenizer.texts_to_sequences(target_sentences)
target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post')

In [13]:
# Create labels for the decoder by shifting the target sequence
# one to the right.
target_labels = np.zeros(target_data.shape)
target_labels[:,0:target_data.shape[1] -1] = target_data[:,1:]

print("Target sequence", target_data[0])
print("Target label", target_labels[0])

Target sequence [ 1 19  3  2  0  0  0  0]
Target label [19.  3.  2.  0.  0.  0.  0.  0.]


In [0]:
source_vocab_size = len(source_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

In [15]:
def decode(encoded, tokenizer):
  for number in encoded:
    if number !=0:
      print ("%d -> %s" % (number, tokenizer.index_word[number]))
      
decode(source_data[0], source_tokenizer)

1 -> <start>
71 -> ve
3 -> .
2 -> <end>


In [0]:
batch_size = 5
dataset = tf.data.Dataset.from_tensor_slices((source_data, target_data, target_labels)).batch(batch_size)

In [17]:
example_batch = next(iter(dataset))
source, target, taget_labels = example_batch
print("Shapes:", source.shape, target.shape, taget_labels.shape)

Shapes: (5, 12) (5, 8) (5, 8)


In [0]:
embedding_size = 32
rnn_size = 64

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self):
    super(Encoder, self).__init__()
    
    self.embedding = tf.keras.layers.Embedding(source_vocab_size,
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)        
    return output, state
  
  def init_state(self, batch_size):
    return tf.zeros((batch_size, rnn_size))

In [20]:
#call encoder
# Create a batch of one sentence
ex_sentence = tf.expand_dims(source_data[0], axis=0)
ex_translation = tf.expand_dims(target_data[0], axis=0)
ex_labels = tf.expand_dims(target_labels[0], axis=0)
print(ex_sentence.shape)

encoder = Encoder()
hidden_state = encoder.init_state(batch_size=1)
print(hidden_state.shape)

output, hidden_state = encoder(ex_sentence, hidden_state)
print(output.shape)

(1, 12)
(1, 64)
(1, 12, 64)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, 
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)

    self.dense = tf.keras.layers.Dense(target_vocab_size)


  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    logits = self.dense(output)
    return logits, state

In [22]:
#call decoder
decoder = Decoder()
decoder_output, decoder_state = decoder(ex_labels, hidden_state)
print(decoder_output.shape)

(1, 8, 1389)


In [23]:
crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def calc_loss(targets, logits):
  mask = tf.math.logical_not(tf.math.equal(targets, 0))
  mask = tf.cast(mask, dtype=tf.int64)
  return crossentropy(targets, logits, sample_weight=mask)

print("Loss", calc_loss(ex_labels, decoder_output))

Loss tf.Tensor(2.7119143, shape=(), dtype=float32)


In [0]:
def translate(idx=None):
  
    if idx == None: 
      idx = np.random.choice(len(sentences))
    
    input_sent = source_data[idx]
    input_sent = tf.expand_dims(input_sent, axis=0)
    
    hidden_state = encoder.init_state(batch_size=1)
    output, hidden_state = encoder(input_sent, hidden_state)
    
    decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    out_words = []
    
    decoder_state = hidden_state

    while True:
      
        decoder_output, decoder_state = decoder(decoder_input, decoder_state)
        decoder_input = tf.argmax(decoder_output, -1)
        word_idx = decoder_input.numpy()[0][0]
        # if we've predicted 0 (which is reserved, usually this will only happen
        # before the decoder is trained, just stop translating and return
        # what we have)
        if word_idx == 0: 
          out_words.append('<end>')
        else:
          out_words.append(target_tokenizer.index_word[word_idx])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
          break
          
    translation = ' '.join(out_words)    
    return sentences[idx][0], sentences[idx][1], translation

In [25]:
input_sent, target_sent, translation = translate()
print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

Input: <start> Eso es enorme . <end>
Target: <start> That s huge . <end>
Translation: release bmw smile release bmw framed became twin they clocks lasts eat dishes yawned hired pushing aim gone hesitant still



In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function # remove this annotation when debugging
def train_step(source_seq, target_seq, target_labels, initial_state):
  
  with tf.GradientTape() as tape:
    encoder_output, encoder_state = encoder(source_seq, initial_state)
    logits, decoder_state = decoder(target_seq, encoder_state)
    loss = calc_loss(target_labels, logits)

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

In [28]:
dataset

<BatchDataset shapes: ((None, 12), (None, 8), (None, 8)), types: (tf.int32, tf.int32, tf.float64)>

In [29]:
EPOCHS = 200

for epoch in range(EPOCHS):
    start = time.time()
  
    en_initial_states = encoder.init_state(batch_size)
    
    for batch, (source_seq, target_seq, target_labels) in enumerate(dataset):
      loss = train_step(source_seq, target_seq, target_labels, en_initial_states)
      elapsed = time.time() - start
    
    if epoch % 10 == 0:
      print("Epoch #%d, Loss %.4f, Time %.2f sec" % (epoch, loss, elapsed))
      input_sent, target_sent, translation = translate()
      print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

Epoch #0, Loss 2.0469, Time 8.25 sec
Input: <start> Yo no vi . <end>
Target: <start> I didn t look . <end>
Translation: i s a a lot . <end>

Epoch #10, Loss 0.9567, Time 6.21 sec
Input: <start> No te muevas . <end>
Target: <start> Be still . <end>
Translation: don t go up . <end>

Epoch #20, Loss 0.6560, Time 6.23 sec
Input: <start> Pero que dia ! <end>
Target: <start> What a day ! <end>
Translation: why a door ! <end>

Epoch #30, Loss 0.4639, Time 6.14 sec
Input: <start> Ya has vuelto . <end>
Target: <start> You re back . <end>
Translation: you re fired . <end>

Epoch #40, Loss 0.3571, Time 7.07 sec
Input: <start> Ellos votaron . <end>
Target: <start> They voted . <end>
Translation: they us boys . <end>

Epoch #50, Loss 0.2772, Time 6.07 sec
Input: <start> Estoy leyendo . <end>
Target: <start> I m reading . <end>
Translation: i m on strike . <end>

Epoch #60, Loss 0.1281, Time 6.06 sec
Input: <start> ¿ Quien te paga ? <end>
Target: <start> Who pays you ? <end>
Translation: who pays yo

In [30]:
#calculate BLEU score
references, hypotheses = [], []

for i in range(len(sentences)):
  input_sent, target_sent, translation = translate()
  references.append(target_sent)
  hypotheses.append("<start> " + translation)
  
results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
print(results)

BLEU(score=35.62104089811024, counts=[22175, 10839, 5774, 2587], totals=[29953, 24953, 19953, 14953], precisions=[74.03265115347378, 43.43766280607542, 28.938004310128804, 17.30087607837892], bp=1.0, sys_len=29953, ref_len=29585)
